In [1]:
import cv2 as cv
from imutils import perspective
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import sudoku_solver as ss
model = tf.keras.models.load_model('digitRecognizer-v2.model')
%matplotlib qt

In [9]:
#Load image and apply gaussian blur
def image_preprocess(path):
    '''Load image, preprocess and apply edge detection.
        Returns the threshholded image, grayscale image and original image'''
    img = cv.imread(path)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    gblur = cv.GaussianBlur(gray, (9,9), 0)
    thresh = cv.adaptiveThreshold(gblur, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY_INV, 11, 2)
    return thresh, gray, img
    # cv.imshow('thresh', thresh)
    # cv.imshow('gblur', gblur)
    # cv.waitKey(0)
    # cv.destroyAllWindows()

In [3]:
#Find the 4-sided contour with largest area and map it to a square of 450x450
def find_grid(thresh, gray, img):
    '''Finds the sudoku grid in the threshholded image, returns grayscale, color versions of the warped image
        and the homography matrix'''
    contours, _ = cv.findContours(thresh, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    max_area = 0
    max_rect = 0
    for contour in contours:
        approx = cv.approxPolyDP(contour, 0.01*cv.arcLength(contour, True), True)
        if len(approx) == 4:
            area = cv.contourArea(approx)
            if area > max_area:
                max_area = area
                max_rect = approx
    pts1 = np.squeeze(max_rect).tolist()
    pts1 = np.float32(pts1)
    pts1 = perspective.order_points(pts1)
    pts2 = np.float32([[0,0], [450, 0], [450,450], [0,450]])
    matrix = cv.getPerspectiveTransform(pts1, pts2)
    result = cv.warpPerspective(gray, matrix, (450, 450))
    color_result = cv.warpPerspective(img, matrix, (450, 450))
    return result, color_result, matrix
#     cv.imshow('result', result)
#     cv.imshow('contours', gray)
#     cv.waitKey(0)
#     cv.destroyAllWindows()

In [4]:
# If the number of white pixels is greater than threshold value then they contain digits
def find_digits(roi, threshold = 100):    
    '''Finds digits in the cropped images, if the number of white pixels is greater 
        than threshold value then they contain digits, returns the digits and their indices'''
    digits = []
    idx = []
    for i in range(81):
        num_white_pixels = np.sum(roi[i] == 255)
        if num_white_pixels > threshold:
            digits.append(cv.bitwise_not(roi[i]).reshape(40,40,1))
            idx.append(i)
    return digits, idx
#       for digit in digits:
#           cv.imshow('', digit)
#           cv.waitKey(0)
#           cv.destroyAllWindows()

In [5]:
def predict_and_solve(digits, idx, plot_predictions = False):
    '''Predicts the digits, forms the sudoku grid and returns it'''
    predictions = []
    for digit in digits:
        predictions.append(int(model.predict_classes(digit.reshape(-1,40,40,1)/255.)))
    grid = ss.build_grid(predictions, idx)
    ss.solve(grid)
    
    if plot_predictions:
        plt.figure()
        for i in range(len(digits)):
            plt.subplot(6, 6, i+1), plt.imshow(digits[i].reshape(40,40))
            plt.title(predictions[i])
            plt.axis('off')
        plt.plot()
    
    return grid

In [6]:
def solve_in_image(color_result, grid, indices, squares):
    '''Using answers in grid it writes the result onto the warped image, indices are required to 
        check whether digit already existed there or not'''
    for i in range(81):
        if i in indices: continue
        row = int(i/9)
        col = i%9
        x, y = squares[i]
        cv.putText(color_result[y+5:y+45, x+5:x+45], f'{grid[row][col]}', (10,30), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv.LINE_AA)

In [10]:
def solve_the_sudoku(path):
    '''Give image path, returns the solved sudoku'''
    thresh, gray, img = image_preprocess(path)
    result, color_result, matrix = find_grid(thresh, gray, img)

    # Get top-left coordinates of each square
    squares = []
    for i in range(9):
        for j in range(9):
            p1 = (j*50, i*50)
            squares.append(p1)

    # Crop the square
    thresh = cv.adaptiveThreshold(result, 255, cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY_INV, 11, 5)
    roi = []
    color_roi = []
    for x, y in squares:
        roi.append(cv.morphologyEx(thresh[y+5:y+45, x+5:x+45], cv.MORPH_OPEN, np.ones((2,2), np.uint8)))
        color_roi.append(color_result[y+5:y+45, x+5:x+45])

    digits, indices = find_digits(roi)
    grid = predict_and_solve(digits, indices)
    solve_in_image(color_result, grid, indices, squares)
    #Reverse warp the result
    answer = img.copy()
    result = cv.warpPerspective(color_result, matrix, img.shape[:2][::-1], answer, cv.WARP_INVERSE_MAP, borderMode=cv.BORDER_TRANSPARENT)
    return answer

In [15]:
path = 'sudoku.jfif'
answer = solve_the_sudoku(path)
cv.imshow('original', cv.imread(path))
cv.imshow('answer', answer)
cv.waitKey(0)
cv.destroyAllWindows()